# ACA-Py & ACC-Py Prover Template

## Copy this template into the root folder of your notebook workspace to get started.

## NOTE: A prover can only respond to proof requests if they have credentials previously stored in their wallet (unless using self-attested attributes). The prover_template works well with the templates in the issue-credential folder.

### Imports

In [ ]:
from aries_cloudcontroller import AriesAgentController
import os
from termcolor import colored

### Initialise the Agent Controller

In [ ]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(
    f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}"
)
agent_controller = AriesAgentController(admin_url, api_key)

### Start a Webhook Server

In [ ]:
webhook_port = os.getenv("WEBHOOK_PORT")
webhook_host = "0.0.0.0"

agent_controller.init_webhook_server(webhook_host, webhook_port)
await agent_controller.listen_webhooks()

print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

## Register Agent Event Listeners

You can see some examples within the webhook_listeners recipe. Copy any relevant cells across and customise as needed.

In [ ]:
listeners = []

# Receive connection messages
def connections_handler(payload):
    state = payload["state"]
    connection_id = payload["connection_id"]
    their_role = payload["their_role"]
    routing_state = payload["routing_state"]

    print("----------------------------------------------------------")
    print("Connection Webhook Event Received")
    print("Connection ID : ", connection_id)
    print("State : ", state)
    print("Routing State : ", routing_state)
    print("Their Role : ", their_role)
    print("----------------------------------------------------------")

    if state == "invitation":
        # Your business logic
        print("invitation")
    elif state == "request":
        # Your business logic
        print("request")

    elif state == "response":
        # Your business logic
        print("response")
    elif state == "active":
        # Your business logic
        print(
            colored(
                "Connection ID: {0} is now active.".format(connection_id),
                "green",
                attrs=["bold"],
            )
        )


connection_listener = {"handler": connections_handler, "topic": "connections"}

listeners.append(connection_listener)


def prover_proof_handler(payload):
    role = payload["role"]
    connection_id = payload["connection_id"]
    pres_ex_id = payload["presentation_exchange_id"]
    state = payload["state"]
    print("\n---------------------------------------------------------------------\n")
    print("Handle present-proof")
    print("Connection ID : ", connection_id)
    print("Presentation Exchange ID : ", pres_ex_id)
    print("Protocol State : ", state)
    print("Agent Role : ", role)
    print("Initiator : ", payload["initiator"])
    print("\n---------------------------------------------------------------------\n")

    if state == "request_received":
        presentation_request = payload["presentation_request"]
        print("Recieved Presentation Request\n")
        print(
            "\nRequested Attributes - Note the restrictions. These limit the credentials we could respond with\n"
        )
        print(presentation_request["requested_attributes"])
    elif state == "presentation_sent":
        print(
            "The Presentation object is a bit overwhelming. Let's look at it in detail\n"
        )

    elif state == "presentation_acked":
        print("Presentation has been acknowledged by the Issuer")


prover_listener = {"topic": "present_proof", "handler": prover_proof_handler}

listeners.append(prover_listener)


agent_controller.register_listeners(listeners)

## Accept Invitation

Copy an invitation object from another agent playing the role inviter (see the inviter_template recipe)

In [ ]:
invitation = {<some agent invitation>}

In [ ]:
auto_accept = "false"
alias = None

invite_response = await agent_controller.connections.receive_invitation(
    invitation, alias, auto_accept
)
connection_id = invite_response["connection_id"]

In [ ]:
# Label for the connection
my_label = None
# Endpoint you expect to recieve messages at
my_endpoint = None

accept_response = await agent_controller.connections.accept_invitation(
    connection_id, my_label, my_endpoint
)

## Send Trust Ping

Once connection moves to response state one agent, either inviter or invitee needs to send a trust ping.

Note: you may not need to run this cell. It depends one of the agents has the ACAPY_AUTO_PING_CONNECTION=true flag set.

In [ ]:
comment = "Some Optional Comment"
message = await agent_controller.messaging.trust_ping(connection_id, comment)

## Optional: Send Proposal

Propose a presentation to a verifier

In [ ]:
# TODO: Example proposal object below

# proposal_object = {
#   "auto_present": true,
#   "comment": "string",
#   "connection_id": "3fa85f64-5717-4562-b3fc-2c963f66afa6",
#   "presentation_proposal": {
#     "@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/present-proof/1.0/presentation-preview",
#     "attributes": [
#       {
#         "cred_def_id": "WgWxqztrNooG92RXvxSTWv:3:CL:20:tag",
#         "mime-type": "image/jpeg",
#         "name": "favourite_drink",
#         "referent": "0",
#         "value": "martini"
#       }
#     ],
#     "predicates": [
#       {
#         "cred_def_id": "WgWxqztrNooG92RXvxSTWv:3:CL:20:tag",
#         "name": "high_score",
#         "predicate": ">=",
#         "threshold": 0
#       }
#     ]
#   },
#   "trace": false
# }

# proposal_response = await agent_controller.proofs.send_proposal(proposal_object)

## Fetch Presentation Records

Before you can present a presentation, you must identify the presentation record which you wish to respond to with a presentation. This could also be done through the present_proof listeners which have access to a presentation record in the payload.

In [ ]:
# Optional Query parameters
verifier_connection_id = connection_id
thread_id = None
state = "request_received"
role = "prover"

proof_records_response = await agent_controller.proofs.get_records(
    verifier_connection_id, thread_id, state, role
)

# We fetch the first record from the response. You may want to customise this further
presentation_record = proof_records_response["results"][0]
presentation_exchange_id = presentation_record["presentation_exchange_id"]

## Search For Available Credentials to Construct Presentation From

The presentation record can be used to query your agents wallet and return all credentials that could be used to construct valid presentation

In [ ]:
# select credentials to provide for the proof
credentials = await agent_controller.proofs.get_presentation_credentials(pres_ex_id)
print(
    "Credentials stored that could be used to satisfy the request. In some situations you applications may have a choice which credential to reveal\n"
)
print(credentials)

credentials_by_reft = {}
revealed = {}
self_attested = {}
predicates = {}


# Note we are working on a friendlier api to abstract this away

if credentials:
    for row in credentials:

        for referent in row["presentation_referents"]:
            if referent not in credentials_by_reft:
                credentials_by_reft[referent] = row

for referent in payload["presentation_request"]["requested_attributes"]:
    if referent in credentials_by_reft:
        revealed[referent] = {
            "cred_id": credentials_by_reft[referent]["cred_info"]["referent"],
            "revealed": True,
        }


print("\nGenerate the proof")
presentation = {
    "requested_predicates": predicates,
    "requested_attributes": revealed,
    "self_attested_attributes": self_attested,
}
print(presentation)

## Send Presentation

A presentation is sent in represent to a presentation record that has previously been created.

In [ ]:
presentation_response = await agent_controller.proofs.send_presentation(
    presentation_exchange_id, presentation
)

## Your Own Business Logic

Now you should have an established, active connection you can write any custom logic you want to engage with protocols with the connection

In [ ]:
## Custom Logic

## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [ ]:
await agent_controller.terminate()